Скрипт вытаскивает из базы ФИАС улицы, населенные пункты и муниципальные районы и делает их них "плоскую" таблицу

Регион добавляю из отдельного файла, м.о. и город определяю по расщепленному коду ФИАС (plaincode).

На выходе получается 3 датасета: муниципальные районы, города и села, улицы.

Это отладочный код, на примере любого региона, лучше брать регион пожирнее, например Московскую область.

Автоматический код в — 02_FIAS_street_auto

Дата: авг. 2020, доработки сентябрь 2021.

**ФИАС перестал выкладывать обновления в формате DBF с сентября 2021.**

In [ ]:
# загружаю нужные библиотеки

# панды
import pandas as pd

# для чтения базы 
from dbfread import DBF

In [ ]:
# оключаю предупреждения

pd.options.mode.chained_assignment = None  # default='warn'

### Федеральная информационная адресная система (ФИАС) ###

Единый российский государственный адресный реестр

Источник: ФНС России

Последняя версия: **https://fias.nalog.ru/Updates**, формат DBF.

### Описание данных: ###

**ACTSTATUS** - Статус последней исторической записи в жизненном цикле адресного объекта:
- 0 – Не последняя;
- 1 - Последняя

**AOGUID** - глобальный уникальный идентификатор адресного объекта

**AOID** - уникальный идентификатор записи (версии). Меняется при изменении адреса.

**AOLEVEL** - уровень адресного объекта
- 1 – регион
- 3 - муниципальный район
- 4 - город
- 6 – населенный пункт
- 7 – улица
- 65 – планировочная структура
- 90 – дополнительные территории (устаревшее)

**AREACODE** - Код района

**AUTOCODE** - Код автономии

**CENTSTATUS** - Статус центра

**CITYCODE** - Код города

**CODE** - Код адресного элемента одной строкой с признаком актуальности из классификационного кода

**CURRSTATUS** - Статус актуальности КЛАДР 4 (последние две цифры в коде)

**ENDDATE** - Окончание действия записи

**FORMALNAME** - Формализованное наименование

**IFNSFL** - Код ИФНС ФЛ

**IFNSUL** - Код ИФНС ЮЛ

**NEXTID** - Идентификатор записи  связывания с последующей исторической записью

**OFFNAME** - Официальное наименование

**OKATO** - ОКАТО

**OKTMO** - ОКТМО

**OPERSTATUS** - Статус действия над записью – причина появления записи (см. OperationStatuses )

**PARENTGUID** - Идентификатор объекта родительского объекта

**PLACECODE** - Код населенного пункта

**PLAINCODE** - Код адресного элемента одной строкой без признака актуальности (последних двух цифр)

**POSTALCODE** - Почтовый индекс

**PREVID** - Идентификатор записи связывания с предыдущей исторической записью. = AOID предыдущей записи.

**REGIONCODE** - Код региона

**SHORTNAME** - Краткое наименование типа объекта

**STARTDATE** - Начало действия записи

**STREETCODE** - Код улицы

**TERRIFNSFL** - Код территориального участка ИФНС ФЛ

**TERRIFNSUL** - Код территориального участка ИФНС ЮЛ

**UPDATEDATE** - Дата  внесения (обновления) записи

**CTARCODE** - Код внутригородского района

**EXTRCODE** - Код дополнительного адресообразующего элемента

**SEXTCODE** - Код подчиненного дополнительного адресообразующего элемента

**LIVESTATUS** - Статус актуальности адресного объекта ФИАС на текущую дату:
- 0 – Не актуальный
- 1 - Актуальный
 
**NORMDOC** - Внешний ключ на нормативный документ

**PLANCODE** - Код элемента планировочной структуры

**CADNUM** - Кадастровый номер

**DIVTYPE** - Тип деления:
- 0 – не определено
- 1 – муниципальное
- 2 – административное

### Кодировка региона ###

Код адреса включает:

СС+РРР+ГГГ+ППП+СССС+УУУУ+ДДДД (или ЗЗЗЗ)+ОООО, где:

- **СС** – код субъекта Российской Федерации  – региона
- **РРР** – код района;
- **ГГГ** – код города;
- **ППП** код населенного пункта;
- **СССС** - код элемента планировочной структуры;
- **УУУУ** - код улицы;
- **ДДДД** (или **ЗЗЗЗ**).  ДДДД  тип и номер здания, сооружения, объекта незавершенного строительства в случае адресации домов. ЗЗЗЗ - номер земельного участка в случае адресации земельных участков;
- **ОООО** - тип и номер помещения в пределах здания, сооружения
Таким образом, каждому уровню классификации соответствует фасет кода.

**Пример 1**

Берем PLAINCODE (тот же CODE, но без двух последних цифр, обозначающих актуальность).

320180000580002 и разбиваем

REGIONCODE - 32 - Брянская область

AREACODE - 018 - Новозыбковский район

CITYCODE - 000 - нет

PLACECODE - 058 - с. Старый Вышков

STREETCODE - 0002 - ул. Ленина

**Пример 2**

320000040000081

REGIONCODE - 32 - Брянская область

AREACODE - 000 - района нет, город областного подчинения

CITYCODE - 004 - Новозыбков

PLACECODE - 000 - none

STREETCODE - 0081 - ул. Ленина

Послений параметр варьируется (STREETCODE / PLANCODE )

**Пример 3**

320050000010122


REGIONCODE - 32 - Брянская область

AREACODE - 005 - Дубровский район

CITYCODE - 000 - none

PLACECODE - 001 - рп Дубровка

PLANCODE - 0122 - гаражи по улице Ленина Блок 1

Содержание датафреймов:

**data** - полный датасет (только актуальные значения, LIVESTATUS - 1)

**work** - рабочие сеты, дополненные и отфильтрованные 

In [ ]:
# читаю файл

dbf = DBF('C:/FIAS/ADDROB91.DBF') # номер соответвует коду региона
data = pd.DataFrame(dbf)

In [ ]:
# фильтрую данные, оставляю только актуальные адреса LIVESTATUS = 1

data = data[data.LIVESTATUS == 1]

In [ ]:
data.shape

In [ ]:
# удаляю возможные дубликаты, т.е. полностью идентичные записи

data = data.drop_duplicates()

In [ ]:
data.shape

In [ ]:
# названия столбцов оригинального файла

all_columns = list(data)
print(all_columns)

#### all_columns, сгруппированные по содержанию: ####

**Статус записи**: 'ACTSTATUS', 'CURRSTATUS', 'LIVESTATUS', 'OPERSTATUS'


**Дата записи**: 'STARTDATE', 'ENDDATE', 'UPDATEDATE'


**Название объекта**: 'FORMALNAME', 'OFFNAME'


**Тип объекта**: 'SHORTNAME', 'AOLEVEL'


фасетный код

**сам код**: 'CODE', 'PLAINCODE', 'REGIONCODE', 'AREACODE', 'CITYCODE', 'PLACECODE', 'PLANCODE', 'STREETCODE',


**Коды ОКАТО, ОКТМО**: 'OKATO', 'OKTMO',


**Почтовый индекс**: 'POSTALCODE'

*Из-за индексов будут дублироваться протяженные улицы, особенно в больших городах. Без надобности их лучше не брать*


**Налоговая**: 'IFNSFL', 'IFNSUL', 'TERRIFNSFL', 'TERRIFNSUL',


'AOGUID', 'AOID', 'AUTOCODE', 'CENTSTATUS', 

'NEXTID', 'PARENTGUID',  

'PREVID',  

'CTARCODE', 'EXTRCODE', 'SEXTCODE', 'CADNUM', 'DIVTYPE'


**код нормативного акта**: 'NORMDOC'

In [ ]:
# вручную выбираю нужные столбцы для рабочего датафрейма

necessary_columns = ['AOLEVEL', 'FORMALNAME', 'OFFNAME', 'SHORTNAME',
                     'PLAINCODE', 'REGIONCODE', 'AREACODE',
                     'CITYCODE', 'PLACECODE', 'PLANCODE', 'STREETCODE',
                     'OKATO', 'OKTMO',
                     #'POSTALCODE',
                     #'NORMDOC', 'AOGUID', 'AOID', 'PREVID'
                    ]

In [ ]:
# оставляю только нужные столбцы в новом фрейме work

work = data[necessary_columns]
work.columns = work.columns.str.lower()

In [ ]:
# меняю типы данных в столбцах для корректного слияния

work = work.astype({'aolevel': 'int32', 'regioncode': 'int32'})

In [ ]:
work.isnull().sum()

**Не верьте нулям!**

In [ ]:
work.shape

In [ ]:
# проверяем столбец на пустые значения

work[work.formalname == ''].shape

In [ ]:
# проверяем столбец на пустые значения

work[work.offname == ''].shape

В августе 2020 я долго возился с данными из-за внезапного глюка: из столбца OFFNAME "выпало" множество значений. Возился пока не догадался посмотреть FORMALNAME.

Глюк исправили, но теперь на всякий случай я всегда сохраняю оба значения и OFF и FORMAL.

**Удаляем лишнее для экономии ресурсов**

Оставляем только записи с уровнем (aolevel) - 1, 3, 4, 6, 7.

65, 90 -- в нашем случае только зашумляют картину.

In [ ]:
work.shape

In [ ]:
# оставляем только нужные записи

work = (work[
    (work.aolevel == 1) |
    (work.aolevel == 3) |
    (work.aolevel == 4) |
    (work.aolevel == 6) |
    (work.aolevel == 7)     
]
)

In [ ]:
work.shape

In [ ]:
work.dtypes

In [ ]:
# смотрю уровни и что там внутри

level_df = work.aolevel.value_counts().rename_axis('level').reset_index(name='counts').sort_values('level')
level_df.head(10)

## Вывожу данные по каждому уровню,

чтобы лучше понять, что внутри

### 3 - уровень района ###

In [ ]:

(work[work.aolevel == 3]
 .shortname.value_counts()
 .rename_axis('element').reset_index(name='number')
 .sort_values('number', ascending=False)
)

### 4 - уровень города ###

In [ ]:
(work[work.aolevel == 4]
 .shortname.value_counts()
 .rename_axis('element').reset_index(name='number')
 .sort_values('number', ascending=False)
)

### 6 - уровень поселка ###

In [ ]:
(work[work.aolevel == 6]
 .shortname.value_counts()
 .rename_axis('element').reset_index(name='number')
 .sort_values('number', ascending=False)
)

### 7 - уровень улицы ###

In [ ]:
(work[work.aolevel == 7]
 .shortname.value_counts()
 .rename_axis('element').reset_index(name='number')
 .sort_values('number', ascending=False)
)

Сейчас в данных еще много лишнего, чистить буду позже, после того, как сведу всё в один файл.

### Добавляем регион ###

In [ ]:
# создаем датафрейм с названиями регионов и кодами ФИАС

reg_code = pd.read_csv('C:/00_Data/General/reg_code.csv')
reg_code.regioncode = reg_code.regioncode.astype('int32')

In [ ]:
reg_code.dtypes

In [ ]:
# добавляем регион

work = work.merge(reg_code, on='regioncode', how='left')

In [ ]:
# проверяем

work.isnull().sum()

In [ ]:
work.sample(3)

### Добавляем муниципальный район ###

In [ ]:
# отбираем районы и города и поселки регионального подчинения

mun_district = work[(
    (work.aolevel == 3)
 |
    (
        (work.aolevel == 4) &
        (work.areacode == '000' )) |
    (
        (work.aolevel == 6) &
        (work.areacode == '000') &
        (work.citycode == '000')
    )
)]

In [ ]:
mun_district.shape

In [ ]:
# Простая функция для разметки данных
# Режет plaincode в соответствии с "муниципальным статусом" объекта

def cut_code(row):
    
    # муниципалитеты
    
    if row.aolevel == 3: # если уровень 3, возвращает только первые 5 символов
        return row.plaincode[0:5]
    
    # городские округа (у г.о. areacode - 000)
    
    elif row.aolevel == 4: # для городских округов — 8
        if row.areacode == '000':
            return row.plaincode[0:8]
        
        else: # но для 
            return row.plaincode[0:5]

    # поселки и улицы
    
    elif row.aolevel == 6 or row.aolevel == 7:
        
        if row.areacode != '000':
            return row.plaincode[0:5]
                
        # поселки которые имеют статус округов
        
        elif row.areacode == '000' and row.citycode == '000':
            return row.plaincode[0:11]            
        
        # все остальное (000 в areacode означает, что объект входит в состав г.о. или с.о.)
        # поэтому берем 8 цифр
        
        else:
            return row.plaincode[0:8]

In [ ]:
# добавляем mun_code в датафрейм для слияния

mun_district['mun_code'] = mun_district.apply(cut_code, axis=1)

In [ ]:
# оставляем нужные столбцы

mun_district = mun_district[['formalname', 'mun_code']]

In [ ]:
# переименовываем

mun_district = mun_district.rename(columns={'formalname': 'mun_district'})

In [ ]:
# добавляем mun_code в датафрейм

work['mun_code'] = work.apply(cut_code, axis=1)

In [ ]:
# объединяем

work = work.merge(mun_district, on='mun_code', how='left')

In [ ]:
work.shape

In [ ]:
# проверка на NaN
# один NaN должен быть в mun_code (это регион)
# некоторое количество может быть в mun_district -- это следствия изменений и ошибок

work.isnull().sum()

## Пропуски в mun_district ##

Часть ошибок можно исправить автоматом

Функция не исправляет все ошибки. Более продуктивно исправить ошибки "вручную", по кодам ОКТМО с https://rosstat.gov.ru/opendata/7708234640-oktmo (надо написать скрипт).

In [ ]:
# заполняем пустые значения
work.fillna('null', inplace=True)

In [ ]:
work[work.mun_district == 'null']

In [ ]:
# примитивная функция для заполнения по ОКТМО (смотрит на другие строки с таким же ОКТМО)
# для городов регионов (Москвы, СПб, Севастополя) функция вместо м.о. добавляет населенные пункты (это неправильно, но пока так)

def add_mun_district(row):
    need_code = row.oktmo
    try:
        new_value = work[(
            (work.oktmo == need_code) &
            (work.mun_district != 'null')
        )].mun_district.to_list()[0]
        
    except:
        new_value = 'null'
        
    return new_value

In [ ]:
# заводим временный датафрейм

temp_md = work[work.mun_district == 'null']

In [ ]:
%%time

# добавляем столбец

temp_md['new_md'] = temp_md.apply(add_mun_district, axis=1)

In [ ]:
temp_md

In [ ]:
# оставляем нужные столбцы для слияния (на всякий случай 4)

temp_md = temp_md[['formalname', 'offname', 'plaincode', 'oktmo', 'new_md']]

In [ ]:
# соединяем

work = work.merge(temp_md, on=['formalname', 'offname', 'plaincode', 'oktmo'], how='left')

In [ ]:
work.head(3)

In [ ]:
# заполняем NaN

work.fillna('null', inplace=True)

In [ ]:
# меняем null в mun_district 

work.loc[(work.mun_district == 'null'), 'mun_district'] = work.new_md

In [ ]:
# проверяем

work[work.mun_district == 'null'].shape

### Добавляем населенный пункт ###

In [ ]:
# создаем датафрейм с городами и поселками

city = work[(
    (work.aolevel == 4) |
    (work.aolevel == 6)
)]

In [ ]:
# оставляем нужные столбцы

city = city[['shortname', 'formalname', 'plaincode']]

In [ ]:
# переименовываем

city = city.rename(columns={'shortname': 'city_type', 'formalname': 'city', 'plaincode': 'city_code'})

In [ ]:
city

In [ ]:
# проверяем

city.shape

In [ ]:
# создаем столбец "код города" для слияния

work['city_code'] = work.plaincode.str[0:11]

In [ ]:
%%time

# объединяем

work = work.merge(city, on='city_code', how='left')

In [ ]:
work.shape

In [ ]:
# пустые значения -- улицы городов "прямого подчинения" (и ошибки с прошлого шага)

work.isnull().sum()

In [ ]:
# заполняем NaN

work.fillna('null', inplace=True)

In [ ]:
# есть некоторое незначительное количество районов (это ошибки ФИАС)

work[work.city == 'null'].mun_district.value_counts()

In [ ]:
work.sample(3)

In [ ]:
# берем city из mun_district

work.loc[(work.city == 'null'), 'city'] = work.mun_district

In [ ]:
work[work.city == 'null'].shape

In [ ]:
work.shape

### Фильтруем данные ###

In [ ]:
# переименовываем formalname в понятный street

work.rename(columns={'formalname': 'street'}, inplace=True)

### Муниципальные районы ###

In [ ]:
# датафрейм муниципальных образований

work_districts = work[(
    (work.aolevel == 3) |
    (
        (work.aolevel == 4) &
        (work.areacode == '000')) |
    (
        (work.aolevel == 6) &
        (work.areacode == '000') &
        (work.citycode == '000'))
)]


# оставляем важные столбцы
work_districts = work_districts[['aolevel', 'shortname', 'mun_district', 'offname',
                                 'region',
                                 'okato', 'oktmo', #'normdoc'
                                ]]

### Города и поселки ###

In [ ]:
#  фильтруем города и поселки

work_cities = work[(
    (work.aolevel == 4) |
    (work.aolevel == 6) |
    (
        (work.aolevel == 3) &
        ((work.shortname == 'г') |
         (work.shortname == 'п'))
    )
)]


# оставляем важные столбцы
work_cities = work_cities[['aolevel', 'city_type', 'city', 'offname',
                           'mun_district', 'region',
                           'okato', 'oktmo', #'normdoc'
                          ]]

### Улицы ###

In [ ]:
#  фильтруем улицы и территории

work_streets = work[(
    (work.aolevel == 7) |
    (work.aolevel == 65)
)]

# оставляем важные столбцы
work_streets = work_streets[['aolevel', 'shortname', 'street', 'offname',
                             'city_type', 'city', 'mun_district', 'region',
                             'okato', 'oktmo', #'normdoc'
                            ]]

In [ ]:
# записываем в файл (чтобы посмотреть, что вышло)

work_streets.to_excel('C:/Users/Boris/Desktop/work_streets.xlsx', encoding='utf-8', index=False)   